## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Library

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

import time

import random
random.seed(777)

from PIL import Image
from torch.autograd import Variable

## GPU Setting

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

print(f'Using PyTorch version: {torch.__version__} / Device: {device}')

Using PyTorch version: 1.8.1+cu101 / Device: cuda


## Hyperparameters

In [ ]:
TRAINPATH = 'drive/MyDrive/Capstone-Design/dataset/AIHub/train' 

VGGNETPATH = 'drive/MyDrive/Capstone-Design/model/AIHub/VGGNet/model_epoch_100_acc_89.pth'
GOOGLENETPATH = 'drive/MyDrive/Capstone-Design/model/AIHub/GoogLeNet/model_epoch_100_acc_97.pth'
RESNETPATH = 'drive/MyDrive/Capstone-Design/model/AIHub/ResNet/model_epoch_100_acc_91.pth'

AUGVGGNETPATH = 'drive/MyDrive/Capstone-Design/model/AIHub_Aug/VGGNet/model_epoch_100_acc_94.pth'
AUGGOOGLENETPATH = 'drive/MyDrive/Capstone-Design/model/AIHub_Aug/GoogLeNet/model_epoch_100_acc_97.pth'
AUGRESNETPATH = 'drive/MyDrive/Capstone-Design/model/AIHub_Aug/ResNet/model_epoch_100_acc_97.pth'

AUG2VGGNETPATH = 'drive/MyDrive/Capstone-Design/model/AIHub_Aug2/VGGNet/model_epoch_100_acc_87.pth'
AUG2GOOGLENETPATH = 'drive/MyDrive/Capstone-Design/model/AIHub_Aug2/GoogLeNet/model_epoch_100_acc_96.pth'
AUG2RESNETPATH = 'drive/MyDrive/Capstone-Design/model/AIHub_Aug2/ResNet/model_epoch_100_acc_95.pth'

AUG3VGGNETPATH = 'drive/MyDrive/Capstone-Design/model/AIHub_Aug3/VGGNet/model_epoch_100_acc_94.pth'
AUG3GOOGLENETPATH = 'drive/MyDrive/Capstone-Design/model/AIHub_Aug3/GoogLeNet/model_epoch_100_acc_96.pth'
AUG3RESNETPATH = 'drive/MyDrive/Capstone-Design/model/AIHub_Aug3/ResNet/model_epoch_100_acc_95.pth'

BATCHSIZE = 32

## Data Load

In [ ]:
transfrom_train = transforms.Compose([
                            transforms.Resize(224),
                            transforms.ToTensor(),
                            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
                            ])

train_dataset = torchvision.datasets.ImageFolder(root=TRAINPATH, transform=transfrom_train)

train_len = int((8/10) * len(train_dataset))
valid_len = len(train_dataset) - train_len

torch.manual_seed(777)
train_data, valid_data = torch.utils.data.dataset.random_split(train_dataset, [train_len, valid_len])

train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCHSIZE, shuffle=True, num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=BATCHSIZE, shuffle=False, num_workers=2)

classes = ('궤양병', '잎곰팡이병', '점무늬병', '황화잎말림바이러스', '아메리카잎굴파리', '청벌레', '정상')

In [ ]:
print('train :', len(train_data))
print('valid :', len(valid_data))

train : 11964
valid : 2991


## Define Model

In [ ]:
class VGG(nn.Module):
    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        
        self.features = features # Convolution layer
        
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096), # 크기 다를시 input 수정
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        ) # FC layer
        
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x) # Convolution 
        x = self.avgpool(x) # avgpool
        x = x.view(x.size(0), -1) #
        x = self.classifier(x) # FC layer
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
                     
    return nn.Sequential(*layers)

cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'], #8 + 3 =11 == vgg11
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'], # 10 + 3 = vgg 13
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'], #13 + 3 = vgg 16
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'], # 16 +3 =vgg 19
    'custom' : [64,64,64,'M',128,128,128,'M',256,256,256,'M']
}

vggnet = VGG(make_layers(cfg['D']), num_classes=7, init_weights=True).to(device)
aug_vggnet = VGG(make_layers(cfg['D']), num_classes=7, init_weights=True).to(device)
aug2_vggnet = VGG(make_layers(cfg['D']), num_classes=7, init_weights=True).to(device)
aug3_vggnet = VGG(make_layers(cfg['D']), num_classes=7, init_weights=True).to(device)

In [ ]:
class ConvBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(ConvBlock, self).__init__()
        
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.ReLU()
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.act(x)
        return x

class InceptionModule(nn.Module):
    
    def __init__(self, in_channels, f_1x1, f_3x3_r, f_3x3, f_5x5_r, f_5x5, f_pp):
        super(InceptionModule, self).__init__()
        
        self.branch1 = nn.Sequential(
            ConvBlock(in_channels, f_1x1, kernel_size=1, stride=1, padding=0)
        )
        
        self.branch2 = nn.Sequential(
            ConvBlock(in_channels, f_3x3_r, kernel_size=1, stride=1, padding=0),
            ConvBlock(f_3x3_r, f_3x3, kernel_size=3, stride=1, padding=1)
        )
        
        self.branch3 = nn.Sequential(
            ConvBlock(in_channels, f_5x5_r, kernel_size=1, stride=1, padding=0),
            ConvBlock(f_5x5_r, f_5x5, kernel_size=5, stride=1, padding=2)
        )
        
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1, ceil_mode=True),
            ConvBlock(in_channels, f_pp, kernel_size=1, stride=1, padding=0)
        )
        
    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        
        return torch.cat([branch1, branch2, branch3, branch4], 1)


class InceptionAux(nn.Module):
    
    def __init__(self, in_channels, num_classes):
        super(InceptionAux, self).__init__()
        
        self.pool = nn.AdaptiveAvgPool2d((4,4))
        self.conv = nn.Conv2d(in_channels, 128, kernel_size=1, stride=1, padding=0)
        self.act = nn.ReLU()
        self.fc1 = nn.Linear(2048, 1024)
        self.dropout = nn.Dropout(0.7)
        self.fc2 = nn.Linear(1024, num_classes)
    
    def forward(self, x):
        x = self.pool(x)
        
        x = self.conv(x)
        x = self.act(x)
    
        x = torch.flatten(x, 1)
        
        x = self.fc1(x)
        x = self.act(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        
        return x


class GoogLeNet(nn.Module):
    
    def __init__(self, num_classes = 10):
        super(GoogLeNet, self).__init__()
      
        self.conv1 = ConvBlock(3, 64, kernel_size=7, stride=2, padding=3)
        self.pool1 = nn.MaxPool2d(3, stride=2, padding=0, ceil_mode=True)
        self.conv2 = ConvBlock(64, 64, kernel_size=1, stride=1, padding=0)
        self.conv3 = ConvBlock(64, 192, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool2d(3, stride=2, padding=0, ceil_mode=True)
        self.inception3A = InceptionModule(in_channels=192,
                                           f_1x1=64,
                                           f_3x3_r=96,
                                           f_3x3=128,
                                           f_5x5_r=16,
                                           f_5x5=32,
                                           f_pp=32)
        self.inception3B = InceptionModule(in_channels=256,
                                           f_1x1=128,
                                           f_3x3_r=128,
                                           f_3x3=192,
                                           f_5x5_r=32,
                                           f_5x5=96,
                                           f_pp=64)
        self.pool4 = nn.MaxPool2d(3, stride=2, padding=0, ceil_mode=True)
        self.inception4A = InceptionModule(in_channels=480,
                                           f_1x1=192,
                                           f_3x3_r=96,
                                           f_3x3=208,
                                           f_5x5_r=16,
                                           f_5x5=48,
                                           f_pp=64)
        self.inception4B = InceptionModule(in_channels=512,
                                           f_1x1=160,
                                           f_3x3_r=112,
                                           f_3x3=224,
                                           f_5x5_r=24,
                                           f_5x5=64,
                                           f_pp=64)
        self.inception4C = InceptionModule(in_channels=512,
                                           f_1x1=128,
                                           f_3x3_r=128,
                                           f_3x3=256,
                                           f_5x5_r=24,
                                           f_5x5=64,
                                           f_pp=64)
        self.inception4D = InceptionModule(in_channels=512,
                                           f_1x1=112,
                                           f_3x3_r=144,
                                           f_3x3=288,
                                           f_5x5_r=32,
                                           f_5x5=64,
                                           f_pp=64)
        self.inception4E = InceptionModule(in_channels=528,
                                           f_1x1=256,
                                           f_3x3_r=160,
                                           f_3x3=320,
                                           f_5x5_r=32,
                                           f_5x5=128,
                                           f_pp=128)
        self.pool5 = nn.MaxPool2d(3, stride=2, padding=0, ceil_mode=True)
        self.inception5A = InceptionModule(in_channels=832,
                                           f_1x1=256,
                                           f_3x3_r=160,
                                           f_3x3=320,
                                           f_5x5_r=32,
                                           f_5x5=128,
                                           f_pp=128)
        self.inception5B = InceptionModule(in_channels=832,
                                           f_1x1=384,
                                           f_3x3_r=192,
                                           f_3x3=384,
                                           f_5x5_r=48,
                                           f_5x5=128,
                                           f_pp=128)
        self.pool6 = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)
        
        self.aux4A = InceptionAux(512, num_classes) 
        self.aux4D = InceptionAux(528, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.pool3(x)
        x = self.inception3A(x)
        x = self.inception3B(x)
        x = self.pool4(x)
        x = self.inception4A(x)
  
        aux1 = self.aux4A(x)
        
        x = self.inception4B(x)
        x = self.inception4C(x)
        x = self.inception4D(x)
  
        aux2 = self.aux4D(x)
        
        x = self.inception4E(x)
        x = self.pool5(x)
        x = self.inception5A(x)
        x = self.inception5B(x)
        x = self.pool6(x)
        x = torch.flatten(x,1)
        x = self.dropout(x)
        x = self.fc(x)
        
        return x, aux1, aux2


googlenet = GoogLeNet(num_classes=7).to(device)
aug_googlenet = GoogLeNet(num_classes=7).to(device)
aug2_googlenet = GoogLeNet(num_classes=7).to(device)
aug3_googlenet = GoogLeNet(num_classes=7).to(device)

In [ ]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = conv1x1(inplanes, planes) #conv1x1(64,64)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes, stride) #conv3x3(64,64)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion) #conv1x1(64,256)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x) # 1x1 stride = 1
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out) # 3x3 stride = stride 
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out) # 1x1 stride = 1
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
        super(ResNet, self).__init__()

        self.inplanes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

resnet = ResNet(Bottleneck, [3, 4, 6, 3], 7, True).to(device)
aug_resnet = ResNet(Bottleneck, [3, 4, 6, 3], 7, True).to(device)
aug2_resnet = ResNet(Bottleneck, [3, 4, 6, 3], 7, True).to(device)
aug3_resnet = ResNet(Bottleneck, [3, 4, 6, 3], 7, True).to(device)

## Load Model

In [ ]:
vggnet.load_state_dict(torch.load(VGGNETPATH))
googlenet.load_state_dict(torch.load(GOOGLENETPATH))
resnet.load_state_dict(torch.load(RESNETPATH))

aug_vggnet.load_state_dict(torch.load(AUGVGGNETPATH))
aug_googlenet.load_state_dict(torch.load(AUGGOOGLENETPATH))
aug_resnet.load_state_dict(torch.load(AUGRESNETPATH))

aug2_vggnet.load_state_dict(torch.load(AUG2VGGNETPATH))
aug2_googlenet.load_state_dict(torch.load(AUG2GOOGLENETPATH))
aug2_resnet.load_state_dict(torch.load(AUG2RESNETPATH))

aug3_vggnet.load_state_dict(torch.load(AUG3VGGNETPATH))
aug3_googlenet.load_state_dict(torch.load(AUG3GOOGLENETPATH))
aug3_resnet.load_state_dict(torch.load(AUG3RESNETPATH))

<All keys matched successfully>

## Performance

In [ ]:
IMAGEPATH = 'drive/MyDrive/Capstone-Design/dataset/AIHub/train/4/11_3.jpg'

trans = transforms.Compose([
                             transforms.Resize(224),
                             transforms.ToTensor(),
                             transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

def image_loader(image_name):
  image = Image.open(image_name)
  image = trans(image).float()
  image = Variable(image, requires_grad=True)
  image = image.unsqueeze(0)
  return image.cuda()

image = image_loader(IMAGEPATH)

In [ ]:
def model_accuracy(model, google=False):
  correct = 0
  total = 0

  class_correct = list(0. for i in range(7))
  class_total = list(0. for i in range(7))

  with torch.no_grad():
      for data in valid_loader:
          images, labels = data
          images = images.to(device)
          labels = labels.to(device)

          if google:
            outputs, aux1, aux2 = model(images)
          else:
            outputs = model(images)

          _, predicted = torch.max(outputs, 1)

          total += labels.size(0)
          correct += (predicted == labels).sum().item()

          c = (predicted == labels).squeeze()
          for i in range(len(labels)):
              label = labels[i]
              class_correct[label] += c[i].item()
              class_total[label] += 1

  print(f'Accuracy of the network on the {len(valid_data)} test images: {(100 * correct)/total:.2f} %')

  for i in range(7):
      print('Accuracy of %s : %.2f %%' % (
          classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def performance(model, google_net=False):
  test = []
  pred = []

  model.eval()

  with torch.no_grad():
    for data in valid_loader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      
      if google_net:
        outputs, aux1, aux2 = model(images)
      else:
        outputs = model(images)

      _, predicted = torch.max(outputs.data, 1)

      for i in range(len(labels)):
        test.append(int(labels[i]))
        pred.append(int(predicted[i]))

  confusion = confusion_matrix(test, pred)
  accuracy = accuracy_score(test, pred)
  precision = precision_score(test, pred, average='macro')
  recall = recall_score(test, pred, average='macro')
  f1 = f1_score(test, pred, average='macro')

  print('Confusion Matrix')
  print(confusion)
  print()
  print(f'accuracy  : {accuracy:.6}')
  print(f'precision : {precision:.6}')
  print(f'recall    : {recall:.6}')
  print(f'f1-score  : {f1:.6}')
    
  

In [ ]:
def single_predict(image, model, google_net=False):
  start = time.time()
  if google_net:
    outputs, aux1, aux2 = model(image)
  else:
    outputs = model(image)
  _, predicted = torch.max(outputs.data, 1)
  end = time.time()
  print(f'predicted {predicted}')
  print(f'time {end - start}')

In [ ]:
def performances(image, model, google=False):
  performance(model, google)
  print()
  model_accuracy(model, google)
  print()
  single_predict(image, model, google)

In [ ]:
print('--------------VGGNet--------------')
performances(image, vggnet, False)
print()

print('--------------Aug VGGNet--------------')
performances(image, aug_vggnet, False)
print()

print('--------------Aug2 VGGNet--------------')
performances(image, aug2_vggnet, False)
print()

print('--------------Aug3 VGGNet--------------')
performances(image, aug3_vggnet, False)

--------------VGGNet--------------
Confusion Matrix
[[  19    0    8    7    4    2    2]
 [   0  118    5    0   10    0   13]
 [   0    1  336    1   22    0   27]
 [   3    0    2   44   15    0   13]
 [   0    2   10    5  624    2   84]
 [   2    0   11    2    4    5    8]
 [   0    1   10    6   37    0 1526]]

accuracy  : 0.893347
precision : 0.807797
recall    : 0.668663
f1-score  : 0.713845

Accuracy of the network on the 2991 test images: 89.33 %
Accuracy of 궤양병 : 45.24 %
Accuracy of 잎곰팡이병 : 80.82 %
Accuracy of 점무늬병 : 86.82 %
Accuracy of 황화잎말림바이러스 : 57.14 %
Accuracy of 아메리카잎굴파리 : 85.83 %
Accuracy of 청벌레 : 15.62 %
Accuracy of 정상 : 96.58 %

predicted tensor([4], device='cuda:0')
time 0.004988431930541992

--------------Aug VGGNet--------------
Confusion Matrix
[[  30    0    1    4    6    0    1]
 [   0  126    1    0    9    1    9]
 [   2    2  369    0    7    1    6]
 [   2    0    2   58    4    0   11]
 [   0    1    9    1  681    1   34]
 [   1    0    6    1    6   1

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy of the network on the 2991 test images: 87.03 %
Accuracy of 궤양병 : 26.19 %
Accuracy of 잎곰팡이병 : 78.77 %
Accuracy of 점무늬병 : 85.01 %
Accuracy of 황화잎말림바이러스 : 74.03 %
Accuracy of 아메리카잎굴파리 : 72.08 %
Accuracy of 청벌레 : 0.00 %
Accuracy of 정상 : 99.18 %

predicted tensor([6], device='cuda:0')
time 0.005243778228759766

--------------Aug3 VGGNet--------------
Confusion Matrix
[[  27    0    7    4    1    0    3]
 [   1  123    7    0    4    0   11]
 [   1   10  351    3    4    0   18]
 [   1    1    3   59    2    0   11]
 [   2    0    7    0  702    4   12]
 [   0    0    1    1    3   20    7]
 [   1    2   13    9    4    0 1551]]

accuracy  : 0.947175
precision : 0.881588
recall    : 0.818684
f1-score  : 0.846327

Accuracy of the network on the 2991 test images: 94.72 %
Accuracy of 궤양병 : 64.29 %
Accuracy of 잎곰팡이병 : 84.25 %
Accuracy of 점무늬병 : 90.70 %
Accuracy of 황화잎말림바이러스 : 76.62 %
Accuracy of 아메리카잎굴파리 : 96.56 %
Accuracy of 청벌레 : 62.50 %
Accuracy of 정상 : 98.16 %

predicted tensor([4

In [ ]:
print('--------------GoogLeNet--------------')
performances(image, googlenet, True)
print()

print('--------------Aug GoogLeNet--------------')
performances(image, aug_googlenet, True)
print()

print('--------------Aug2 GoogLeNet--------------')
performances(image, aug2_googlenet, True)
print()

print('--------------Aug3 GoogLeNet--------------')
performances(image, aug3_googlenet, True)

--------------GoogLeNet--------------
Confusion Matrix
[[  40    1    0    1    0    0    0]
 [   0  137    1    0    3    2    3]
 [   1    0  384    0    2    0    0]
 [   0    0    1   72    0    1    3]
 [   0    3    2    0  709    4    9]
 [   1    1    1    0    1   21    7]
 [   0    0    3    3    8    0 1566]]

accuracy  : 0.979271
precision : 0.937273
recall    : 0.920097
f1-score  : 0.928199

Accuracy of the network on the 2991 test images: 97.93 %
Accuracy of 궤양병 : 95.24 %
Accuracy of 잎곰팡이병 : 93.84 %
Accuracy of 점무늬병 : 99.22 %
Accuracy of 황화잎말림바이러스 : 93.51 %
Accuracy of 아메리카잎굴파리 : 97.52 %
Accuracy of 청벌레 : 65.62 %
Accuracy of 정상 : 99.11 %

predicted tensor([4], device='cuda:0')
time 0.015490055084228516

--------------Aug GoogLeNet--------------
Confusion Matrix
[[  40    0    1    0    0    0    1]
 [   0  138    2    0    1    1    4]
 [   0    0  386    0    0    0    1]
 [   0    0    1   73    0    1    2]
 [   0    2    4    0  716    0    5]
 [   1    1    0    1   

In [ ]:
print('--------------ResNet--------------')
performances(image, resnet, False)
print()

print('--------------Aug ResNet--------------')
performances(image, aug_resnet, False)
print()

print('--------------Aug2 ResNet--------------')
performances(image, aug2_resnet, False)
print()

print('--------------Aug3 ResNet--------------')
performances(image, aug3_resnet, False)

--------------ResNet--------------
Confusion Matrix
[[  32    1    0    3    2    0    4]
 [   1  117    0    0    7    1   20]
 [   0    0  337    4   22    0   24]
 [   0    0    2   64    0    1   10]
 [   1    0    7    1  655    2   61]
 [   0    0    3    1    9    9   10]
 [   5    2   11    6   21    1 1534]]

accuracy  : 0.918756
precision : 0.860263
recall    : 0.774049
f1-score  : 0.805398

Accuracy of the network on the 2991 test images: 91.88 %
Accuracy of 궤양병 : 76.19 %
Accuracy of 잎곰팡이병 : 80.14 %
Accuracy of 점무늬병 : 87.08 %
Accuracy of 황화잎말림바이러스 : 83.12 %
Accuracy of 아메리카잎굴파리 : 90.10 %
Accuracy of 청벌레 : 28.12 %
Accuracy of 정상 : 97.09 %

predicted tensor([4], device='cuda:0')
time 0.016103029251098633

--------------Aug ResNet--------------
Confusion Matrix
[[  37    1    0    2    1    0    1]
 [   2  135    4    0    2    0    3]
 [   0    3  377    0    3    1    3]
 [   1    0    0   70    0    1    5]
 [   0    2    3    0  714    2    6]
 [   0    1    2    1    4   2